# Mean Reciprocal Rank (MRR)を実装
MRRは以下のように定式化される．セッションの番号を表すラベルを$j$とし，計$Q$個のセッションがあるものとする．\
また$j$番目のセッションで購入された商品のラベルを$t_j$とする．\
いま，$j$番目のセッションに対する商品$t_j$のレコメンド順位が${\rm rank}_j$であったとき，MRRは
$$
{\rm MRR} = \dfrac{1}{|Q|} \sum_{j=1}^{|Q|} \dfrac{1}{{\rm rank}_j}
$$
のように定義される．ただし，商品$t_j$がレコメンドに入っていない場合は${\rm rank}_j \to \infty$とみなし，$\dfrac{1}{{\rm rank}_j} = 0$として計算する．

# セットアップ
ほとんど使わない

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import re
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# MRRを計算する関数の定義

In [2]:
# df_targetは正解ラベル，列には["session_id", "item_id"]が必要．
# df_predはsubmitと同様の順位表，列には["session_id", "item", "rank"]が必要．
def mean_reciprocal_rank(df_target, df_pred):
    df_MRR = pd.merge(df_train_purchases,df_pred, how = "left", on = ["session_id", "item_id"]) # 各セッションごとに，購入された商品の順位を記録
    df_MRR["reciprocal_rank"] = 1.0 / df_MRR["rank"] # reciprocal rankを計算
    df_MRR["reciprocal_rank"].fillna(0.0, inplace = True) # 順位がない場合はreciprocal rankは0になる
    return df_MRR["reciprocal_rank"].mean() # 平均値がMRRになる

# 簡単な例で出力を確認

## 正解ラベルの読み込み

In [3]:
num_sample = 10000 # 今回使用するサンプル数

# 正解ラベルの読み込み
path_data = r'C:\Users\yota-\Desktop\study\mystudy\recsys\dressipi_recsys2022\data'
df_train_purchases = pd.read_csv(path_data + r"\train_purchases.csv").sample(n = num_sample) # 一部だけをサンプル
display(df_train_purchases)

,session_id,item_id,date
545792,2423262,8606,2020-04-19 00:28:39.527
524265,2327830,4193,2021-02-27 22:25:46.488
264263,1176411,6089,2021-05-10 13:00:53.459
95837,427903,25846,2020-12-04 21:06:36.308
355140,1578692,8004,2020-08-24 00:34:56.907
...,...,...,...
79050,353574,14525,2020-12-09 10:05:10.26
828526,3678740,4504,2020-08-29 20:10:02.116
747788,3320262,17296,2020-11-15 00:34:17.053
275969,1228549,551,2021-05-26 15:27:35.699


## 入力データから単純な出現回数の順位表を作る

In [4]:
# 入力データの読み込み
df_train_sessions = pd.read_csv(path_data + r"\train_sessions.csv")

# 入力データ中の出現回数のみから順位表を作る
num_pred = 100
df_rank = df_train_purchases.groupby("item_id").size().to_frame().rename(columns = {0:"count"})
df_rank = df_rank.sort_values(by = "count", ascending = False).head(num_pred)

df_rank["item_id"] = df_rank.index.values
df_rank["rank"] = range(1, num_pred+1, 1)
df_rank.drop(axis = 1, columns = ["count"], inplace = True)
display(df_rank)

,item_id,rank
item_id,,
8060,8060,1
26853,26853,2
2447,2447,3
4193,4193,4
8622,8622,5
...,...,...
15403,15403,96
3233,3233,97
21890,21890,98


## 単純な順位表のみから予測結果を作る

In [5]:
# pandasはcross_joinがないらしい(マジ？)
def cross_join(df_a, df_b, common_key=None):
    # 同じ列名を持つ場合にはうまく動かないので注意
    df_a['tmp'] = 1
    df_b['tmp'] = 1
    return pd.merge(df_a, df_b, how='outer').drop("tmp", axis = 1)

# 順位表に基づいてpredictionする関数を作成
def predict_by_rank_table(df_test):
    session_id_list = df_test["session_id"].unique()
    df_pred = pd.DataFrame(columns = ["session_id"])
    df_pred["session_id"] = session_id_list
    return cross_join(df_pred, df_rank)

df_pred = predict_by_rank_table(df_train_purchases)
display(df_pred)

,session_id,item_id,rank
0,2423262,8060,1
1,2423262,26853,2
2,2423262,2447,3
3,2423262,4193,4
4,2423262,8622,5
...,...,...,...
999995,3667335,15403,96
999996,3667335,3233,97
999997,3667335,21890,98
999998,3667335,18623,99


順位表をそのまま出力した場合のスコアが0.16程度だったので，割とあっていそう．何度か試しても大きなずれはない．

In [6]:
# df_train_purchasesが正解ラベル，df_predが予測の順位表
mean_reciprocal_rank(df_train_purchases, df_pred)

0.016191433706531005